In [29]:
import os, fnmatch
import sys
import numpy as np

import imageio
import scipy.ndimage
from skimage import img_as_uint

import pydicom
import nibabel as nib
import matplotlib.pyplot as plt
import time

In [30]:
train_dir = "train_data"
valid_dir = "validation_data"
test_dir = "test_data"

png_dir = "images"
labels_dir = "labels"

train_png_dir = os.path.join(train_dir, png_dir)
train_labels_dir = os.path.join(train_dir, labels_dir)

test_png_dir = os.path.join(test_dir, png_dir)
test_labels_dir = os.path.join(test_dir, labels_dir)

valid_png_dir = os.path.join(valid_dir, png_dir)
valid_labels_dir = os.path.join(valid_dir, labels_dir)

In [31]:
##############################################################
##################### Sunnybrook Data ########################
##############################################################

training_dicom_dir = "original_data/sunnybrook_data/Sunnybrook_Images"
training_labels_dir = "original_data/sunnybrook_data/Sunnybrook_Contours"

patients_for_test_sb = ("SC-HF-I-01", "SC-HF-I-40", "SC-HF-NI-03", "SC-HYP-01", "SC-N-02")
patients_for_valid_sb = ("SC-HF-NI-33", "SC-HYP-08")

In [32]:
for root, dirs, files in os.walk(training_labels_dir):
    for file_o in files:
        if file_o.endswith("-ocontour-manual.txt"):
            try:
                prefix, _ = os.path.split(root)
                prefix, _ = os.path.split(prefix)
                _, patient = os.path.split(prefix)


                file_i = file_o.strip("-ocontour-manual.txt") + "-icontour-manual.txt"

                file_fn = file_o.strip("-ocontour-manual.txt") + ".dcm"
                #print(file_fn)

                #print(patient)
                dcm = pydicom.read_file(os.path.join(training_dicom_dir, patient, "DICOM" ,file_fn))
                dcm_new = dcm.pixel_array / (np.max(dcm.pixel_array))
                
                dcm_new = (dcm_new*255).astype(np.uint8)
                
                #print(dcm.pixel_array.shape)
                #img = np.concatenate((dcm.pixel_array[...,None], dcm.pixel_array[...,None], dcm.pixel_array[...,None]), axis=2)
                labels_o = np.zeros_like(dcm.pixel_array)
                labels_i = np.zeros_like(dcm.pixel_array)

                #print(img.shape)
                #print(labels.shape)
                with open(os.path.join(root, file_o)) as labels_f:
                    for line in labels_f:
                        x, y = line.split(" ")
                        labels_o[int(float(y)), int(float(x))] = 128
                labels_o = scipy.ndimage.binary_fill_holes(labels_o)

                
                
                
                with open(os.path.join(root, file_i)) as labels_f:
                    for line in labels_f:
                        x, y = line.split(" ")
                        labels_i[int(float(y)), int(float(x))] = 128
                labels_i = scipy.ndimage.binary_fill_holes(labels_i)
                
                labels_o = (labels_o*255).astype(np.uint8)
                labels_i = (labels_i*255).astype(np.uint8)
                
                labels_new = np.zeros_like(labels_o)
                labels_new += labels_o
                labels_new = np.abs(labels_new-labels_i)
                
                if patient.startswith(patients_for_test_sb):
                    imageio.imwrite(os.path.join(test_png_dir, patient + "-" + file_fn + ".png"), dcm_new)
                    imageio.imwrite(os.path.join(test_labels_dir, patient + "-" + file_fn + ".png"), labels_new)
                    
                elif patient.startswith(patients_for_valid_sb):
                    imageio.imwrite(os.path.join(valid_png_dir, patient + "-" + file_fn + ".png"), dcm_new)
                    imageio.imwrite(os.path.join(valid_labels_dir, patient + "-" + file_fn + ".png"), labels_new)
                
                else:
                    imageio.imwrite(os.path.join(train_png_dir, patient + "-" + file_fn + ".png"), dcm_new)
                    imageio.imwrite(os.path.join(train_labels_dir, patient + "-" + file_fn + ".png"), labels_new)
            
            except Exception as e:
                print(e)

In [33]:
##############################################################
######################### ACDC Data ##########################
##############################################################

training_dir = "original_data/acdc_data/"

patients_for_test_acdc = (
"patient003","patient008","patient015","patient024","patient035",
"patient040","patient051","patient066","patient076","patient087")

patients_for_valid_acdc = (
"patient005","patient099","patient045","patient074")


In [ ]:
# no 038, 057, 085, 089, 100,
# ED=1, ES>

In [34]:
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

def padding_image_and_standardize(img):
    (dim1,dim2,dim3) = img.shape
    new_img = (img - np.min(img))/(np.max(img)-np.min(img))
    new_image = np.zeros( shape=(256,256,dim3) )
    new_image[0:dim1,0:dim2,0:dim3] = new_img
    return new_image

def padding_mask_and_select_ring(img):
    (dim1,dim2,dim3) = img.shape
    new_img = 1.*(img==2)
    new_image = np.zeros( shape=(256,256,dim3) )
    new_image[0:dim1,0:dim2,0:dim3] = new_img
    return new_image

def str_fun(i):
    if i<10:
        return "00"+str(i)
    elif i>=10 and i<100:
        return "0"+str(i)
    else:
        return str(i)
    

x_train = []
y_train = []

for i in range(1,101):
    print(i)
    path_iniz = training_dir
    patient = "patient" + str_fun(i)
    path_iniz += patient
    patient+="_frame*"
    result = find(patient, path_iniz)
    result_other=[]
    
    if patient.startswith(patients_for_test_acdc):
        path_img = test_png_dir         
        path_labels = test_labels_dir
    elif patient.startswith(patients_for_valid_acdc):
        path_img = valid_png_dir         
        path_labels = valid_labels_dir
    else:
        path_img = train_png_dir         
        path_labels = train_labels_dir

    patient = "patient" + str_fun(i)
    
    for name in result:
        if fnmatch.fnmatch(name, "*frame01*"):
            if fnmatch.fnmatch(name, "*_gt.nii.gz"):
                img = nib.load(name)
                if(np.max(img.shape)<257):
                    y = padding_mask_and_select_ring(img.get_fdata())
                    for k in range(y.shape[2]):
                        name_slice = "slice" + str_fun(k)
                        yy = (y[:,:,k]*255).astype(np.uint8)
                        imageio.imwrite(os.path.join(path_labels, patient + "-ED-" + name_slice + ".png"), yy)
                            
            else:
                img = nib.load(name)
                if(np.max(img.shape)<257):
                    x = padding_image_and_standardize(img.get_fdata())
                    for k in range(x.shape[2]):
                        name_slice = "slice" + str_fun(k)
                        xx = (x[:,:,k]*255).astype(np.uint8)
                        imageio.imwrite(os.path.join(path_img, patient + "-ED-" + name_slice + ".png"), xx)
                    
        else:
            result_other.append(name)
            
    for name in result_other:
        if fnmatch.fnmatch(name, "*_gt.nii.gz"):
            img = nib.load(name)
            if(np.max(img.shape)<257):
                y = padding_mask_and_select_ring(img.get_fdata())
                for k in range(y.shape[2]):
                    name_slice = "slice" + str_fun(k)
                    yy = (y[:,:,k]*255).astype(np.uint8)
                    imageio.imwrite(os.path.join(path_labels, patient + "-ES-" + name_slice + ".png"), yy)
        else:
            img = nib.load(name)
            if(np.max(img.shape)<257):
                x = padding_image_and_standardize(img.get_fdata())
                for k in range(x.shape[2]):
                    name_slice = "slice" + str_fun(k)
                    xx = (x[:,:,k]*255).astype(np.uint8)
                    imageio.imwrite(os.path.join(path_img, patient + "-ES-" + name_slice + ".png"), xx)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
